Uppdrag 1

In [385]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px 

# read in the file, found a lot of NaN
df_file = './Data/riket2023_åk9_np.xlsx'

# reads all sheets into a sictionary of DataFrames
# got information on https://www.geeksforgeeks.org/how-to-read-excel-multiple-sheets-in-python-pandas/
# skipped all NAN rows with setting the header to 8
df_sheets = pd.read_excel(df_file, header = 8, sheet_name=None)
sheets = ['Engelska', 'Matematik', 'Svenska', 'Svenska som andraspråk']

In [ ]:
# rename all the columns
df_sheets = {
    sheet: df.rename(columns={
        'Riket': 'Plats',
        'Typ av huvudman': 'Huvudman',
        'Totalt': 'Totalt (A-F)',
        'Flickor': 'Flickor (A-F)',
        'Pojkar': 'Pojkar (A-F)',
        'Totalt ': 'Totalt (A-E)',
        'Flickor ': 'Flickor (A-E)',
        'Pojkar ': 'Pojkar (A-E)',
        'Totalt  ': 'Totalt (Poäng)',
        'Flickor  ': 'Flickor (Poäng)',
        'Pojkar  ': 'Pojkar (Poäng)'
    }) for sheet, df in df_sheets.items()
}

# Kontrollera info för Engelska
df_sheets["Engelska"].info()


In [387]:
# tried to create different colours for every bar in the chart
# found info on https://python-graph-gallery.com/3-control-color-of-barplots/

# create a function so that I can assign the colors
def bar_colour(subjects_df):
    colour_huvudman = {
    'Skolverket' : 'purple',
    'Kommunal': 'green',
    'Enskild': 'blue',
    'Samtliga': 'orange'
}
    return[colour_huvudman.get(huvudman, 'black') for huvudman in subjects_df["Huvudman"]]

In [ ]:
# since "Svenska som andraspråk" only contains two ".." need to convert to NaN
# with replace, then fill the column with 0 so it will look better in the chart
df_sheets["Svenska som andraspråk"]["Totalt (Poäng)"] = df_sheets["Svenska som andraspråk"]["Totalt (Poäng)"].replace("..", np.nan)
df_sheets["Svenska som andraspråk"]["Totalt (Poäng)"] = df_sheets["Svenska som andraspråk"]["Totalt (Poäng)"].fillna(0)

In [ ]:
# plot the barchart, need to use subplots 
# categorys = the different subjects for the total points in the total subjects
# english, swedish, maths and swedish as second language
fig, axs = plt.subplots(1, 4, dpi = 120, figsize =(15, 4))

# main title
fig.suptitle("Totala poäng för huvudmännen", fontsize=15) 

# first plot, english
axs[0].bar(df_sheets["Engelska"]['Huvudman'], 
    df_sheets["Engelska"]['Totalt (Poäng)'],
    color = bar_colour(df_sheets["Engelska"])
)
axs[0].set_title("Engelska", fontsize = 10)
axs[0].set_xticks([0,1,2,3])
axs[0].tick_params(axis='x', rotation=90)

# second plot, math
axs[1].bar(df_sheets["Matematik"]['Huvudman'], 
    df_sheets["Matematik"]['Totalt (Poäng)'],
    color = bar_colour(df_sheets["Engelska"])
)
axs[1].set_title("Matematik", fontsize = 10)
axs[1].set_xticks([0,1,2,3])
axs[1].tick_params(axis='x', rotation=90)

# third plot for swedish
axs[2].bar(df_sheets["Svenska"]['Huvudman'], 
    df_sheets["Svenska"]['Totalt (Poäng)'],
    color = bar_colour(df_sheets["Engelska"])
)
axs[2].set_title("Svenska", fontsize = 10)
axs[2].set_xticks([0,1,2,3])
axs[2].tick_params(axis='x', rotation=90)

# fourth plot for swedish as second language
axs[3].bar(df_sheets["Svenska som andraspråk"]['Huvudman'], 
    df_sheets["Svenska som andraspråk"]['Totalt (Poäng)'],
    color = bar_colour(df_sheets["Engelska"])
)
axs[3].set_title("Svenska som andraspråk", fontsize = 10)
axs[3].set_xticks([0,1,2,3])
axs[3].tick_params(axis='x', rotation=90)

plt.savefig("Visualiseringar/Uppgift_1.png")
plt.show()

Uppgift 2

In [ ]:
# read in the specific Tabell from the excel file
# lots of NaN values
df_tabell_sheet = pd.read_excel("./Data/betyg_o_prov_riksnivå.xlsx", header= 7, sheet_name="Tabell 1B")
df_tabell_sheet.head()

In [ ]:
# let's rename the columns first
df_tabell_sheet.rename(columns ={
        'Unnamed: 0': 'Kategorier',
        'Totalt': 'Totalt (16 Ämnen)', 
        'Flickor': 'Flickor (16 Ämnen)', 
        'Pojkar': 'Pojkar (16 Ämnen)', 
        'Totalt.1': 'Totalt (17 Ämnen)', 
        'Flickor.1': 'Flickor (17 Ämnen)', 
        'Pojkar.1': 'Pojkar (17 Ämnen)', 
        'Totalt.2': 'Total (%)', 
        'Flickor.2': 'Flickor (%)', 
        'Pojkar.2': 'Pojkar (%)', 
    }, inplace = True)

df_tabell_sheet.head()

In [392]:
# filter out the years from the Kategori tabell
# I use the isin() method to choose the rows based on multiple values
# got info on https://www.w3schools.com/python/pandas/ref_df_isin.asp
df_years_filtered = df_tabell_sheet.loc[df_tabell_sheet['Kategorier'].isin(['2018/19', '2019/20', '2020/21',
       '2021/22', '2022/23'])]

In [ ]:
# plotly line diagram for students with no passing grade
# found info on https://plotly.com/python/plotly-express/
fig = px.line(df_years_filtered , 
            x = "Kategorier", 
            y = ["Total (%)", "Flickor (%)", "Pojkar (%)"], 
            title = "Andel elever som saknar godkänt betyg",
            labels = {"value": "Andel (%)", "variable": "Kategorier", "Kategorier": "Läsår"}, 
            line_shape="spline", render_mode="svg", 
            markers = True)

fig.write_html("Visualiseringar/Uppgift_2a.html")
fig.show() 

In [ ]:
# plotly diagram for meritvärde för 16 ämnen, for pojkar, flickor och totalt
fig = px.line(df_years_filtered , x = "Kategorier", y = ["Totalt (16 Ämnen)", "Flickor (16 Ämnen)", "Pojkar (16 Ämnen)"], 
title = "Meritvärde för 16 ämnen",
labels = {"value": "Meritvärde", "variable": "Kategorier", "Kategorier": "Läsår",}, 
line_shape="spline", 
render_mode="svg",
markers = True)

fig.write_html("Visualiseringar/Uppgift_2b.html")
fig.show()

Bonus

In [395]:
df_huvudman_data = df_tabell_sheet[df_tabell_sheet['Kategorier'].isin(['Kommun', 'Enskild'])]

In [ ]:
# bar chart for the Huvudman
fig = px.bar(df_huvudman_data, 
            x = "Kategorier", 
            y=["Total (%)", "Flickor (%)", "Pojkar (%)"], 
            title="Andel elever som saknar godkänt betyg",
            labels={"value": "Andel %", "variable": "Kategorier", "Kategorier": "Huvudman"},
            barmode = 'group')

fig.write_html("Visualiseringar/Uppgift_2c_1.html")
fig.show()

In [397]:
df_background_data = df_tabell_sheet[df_tabell_sheet['Kategorier'].isin(['Elever med svensk bakgrund', 'Elever med utländsk bakgrund'])]

In [ ]:
 # a bar chart for students backgrounds
fig = px.bar(df_background_data, 
x = "Kategorier", 
y = ["Totalt (17 Ämnen)", "Flickor (17 Ämnen)", "Pojkar (17 Ämnen)"], 
title = "Meritvärde för 17 ämnen baserat på elevers bakgrund",
labels = {"value": "Meritvärde", "variable": "Kategorier", "Kategorier": ""}, 
barmode = 'group')

fig.write_html("Visualiseringar/Uppgift_2c_2.html")
fig.show()

In [399]:
df_parents_data = df_tabell_sheet[df_tabell_sheet['Kategorier'].isin(['Förgymnasial utbildning', 'Gymnasial utbildning', 'Eftergymnasial utbildning'])]

In [ ]:
# a bar char on the parents education
fig = px.bar(df_parents_data,
x = "Kategorier", 
y = ["Totalt (16 Ämnen)", "Flickor (16 Ämnen)", "Pojkar (16 Ämnen)"], 
title = "Meritvärde för 16 ämnen baserat på föräldrarnas utbildning",
labels = {"value": "Meritvärde", "variable": "Kategorier", "Kategorier": "Utbildning"}, 
barmode = 'group')

fig.write_html("Visualiseringar/Uppgift_2c_3.html")
fig.show()